In [ ]:
import pandas as pd
import numpy as np


In [44]:
df = pd.read_excel("./dataset.xlsx")

In [45]:
np.random.seed(21)

pd.set_option("display.max_columns", None)      # zobrazenie vsetkych stlpcov
pd.set_option("display.width", 200)             # rozsirenie sirky vypisu
pd.set_option("display.max_rows", 100)          # viacej riadkov

In [46]:
df.columns = (
    df.columns
    .str.replace(r"[.: ]", "_", regex=True)
    .str.replace(r"_+", "_", regex=True)
    .str.strip("_")
)

## Spracovanie atributov

In [47]:
print(df["D0mm_Prog"].value_counts(), "\n")
print(df["Ecc_Zone"].value_counts(), "\n")

D0mm_Prog
0     3373
15       2
18       1
28       1
13       1
17       1
14       1
Name: count, dtype: int64 

Ecc_Zone
Zone 8mm(Dia)    3380
Name: count, dtype: int64 



In [48]:
attrib_want_remove: list[str] = ["Unnamed_0_1", "Unnamed_0", "Exam_name", "Pupil_Pos_X",
                                 "Pupil_Pos_Y", "CXL_date", "CXL_YES", "DatasetID",
                                 "Before_After_CXL", "D0mm_Prog", "Ecc_Zone", "obs_ID",
                                 "LC_id"
                                 ]

df = df.drop(attrib_want_remove,
             axis=1
             )

### Preco prave tieto stlpce?
- `Unnamed_0_1`: nemame o nom ziadne informacie v povodnej dokumentacii (nevieme co data znamenaju)
- `Unnamed_0`: nemame o nom ziadne informacie v povodnej dokumentacii (nevieme co data znamenaju)
- `Exam_name`: nemame o nom ziadne informacie v povodnej dokumentacii (nevieme co data znamenaju)
- `Pupil_Pos_X`: velmi vela zaznamov nema tieto hodnoty (zistit %)
- `Pupil_Pos_Y`: velmi vela zaznamov nema tieto hodnoty (zistit %)
- `CXL_date`: nemyslim si, ze tento datum je potrebny v mojej situacii, lebo mi nedava (aspon v ton nevidim) dolezitu informaciu
- `CXL_YES`: nemyslim si, ze mat informaciu ci pacient mal alebo nikdy nemal spraveny CLX
- `DatasetID`: neviem si predstavit, ze ako by mi informacie ohladom data tychto dat pomohla urcit vystup
- `Before_After_CXL`: neviem ako by mi informacia ohladom toho, ci bolo meranie vykonane pred alebo po CXL pomohla predikovat hodnotu
- `D0mm_Prog`: Stlpec bol odstraneny lebo nad 99% v nom boli rovnake hodnoty
- `Ecc_Zone`: Stlpec bol odstateny lebo obsahoval len tu istu hodnotu
- `obs_ID`: nemyslim si, ze by mi pomohlo observation ID
- `LC_id`: nemyslim si, ze by mi pomholo ID for observation

In [49]:
# V zadani som mal informaciu, ze ak je hodnota NaN, tak by mala byt 0
df["Rubbing"] = df["Rubbing"].fillna(0)


In [50]:
nan_percent = df.isna().mean() * 100
columns_over_20: list[str] = nan_percent[nan_percent > 20]  # type: ignore
columns_over_20 = columns_over_20.index.tolist()            # type: ignore
columns_over_20.remove("referred_CXL")

print(columns_over_20)

df = df.drop(columns_over_20,
             axis=1
             )

['kMaxDiff_N', 'Crit_kMax_N', 'PachyMinDiff', 'PachyMinPercentChange_Exams', 'PachyMinPercentChange_Start', 'Crit_PachyMin_N', 'Crit_Both_N']


In [51]:
df.isna().sum()[df.isna().sum() > 0]

Gender            38
PachyMin           1
D8mm_Prog          7
D10mm_Prog         9
referred_CXL    2547
dtype: int64

#### Spracovanie Stlpcov s NaN hodnotamy

In [52]:
gender_check = (
    df
    .groupby("PateID")["Gender"]
    .agg(
        total_records="size",
        nan_gender=lambda x: x.isna().sum()
    )
    .query("nan_gender > 0")
)

print(gender_check)

        total_records  nan_gender
PateID                           
65                  2           2
205                 2           2
320                 2           2
321                 3           3
757                 3           3
856                 9           9
875                 5           5
904                 2           2
911                 5           5
992                 5           5


- 38 hodnot bolo v stlpci "Gender". Chcel som overit, ze ci nahodou nedoslo ku chybe pri vkladani dat (jeden pacient mohol mat 3 merania a v jednom z nich mohlo byt Gender=Nan, a v inom Gender=1). Po overeni som zistil, ze vsetci pacienti pre ktorych sa naslo aspon jeden zaznam Gender=Nan maju vsetky zaznamy s Nan. Preto si mozem dovolit odstranit tieto zaznamy

In [53]:
bad_ids = gender_check.index
df = df[~df["PateID"].isin(bad_ids)]

In [54]:
d10_prog_check = (
    df
    .groupby("PateID")["D10mm_Prog"]
    .agg(
        total_records="size",
        d10_prog=lambda x: x.isna().sum()
    )
    .query("d10_prog > 0")
)

print(d10_prog_check)

d8_prog_check = (
    df
    .groupby("PateID")["D8mm_Prog"]
    .agg(
        total_records="size",
        d8_prog=lambda x: x.isna().sum()
    )
    .query("d8_prog > 0")
)

print(d8_prog_check)

        total_records  d10_prog
PateID                         
152                 8         1
177                 6         1
821                 7         1
826                 7         1
916                10         1
1017               11         2
1026               12         2
        total_records  d8_prog
PateID                        
821                 7        1
826                 7        1
916                10        1
1017               11        2
1026               12        2


- Kedze chyba len jeden zaznam pri niekotrych atributov, nebude problem dopocitat ho pomocou priemeru. Taktiez, tyto pacienti maju viac zaznamov, cize to nebude velky problem

In [55]:
cols = ["D10mm_Prog", "D8mm_Prog"]

for col in cols:
    df[col] = (
        df
        .groupby("PateID")[col]
        .transform(lambda x: x.fillna(x.mode().iloc[0]))
    )


#### Odstranenie stlpcov, v kotrych chyba viac ako 20%
- Malo dat, nevieme dopocitat = zbytocne stlpce, nedaju sa pouzit

In [56]:
random_patient = (
    df["PateID"]
    .dropna()
    .unique()
)

random_patient = np.random.choice(
    random_patient,
    size=10,
    replace=False
)

df_sample = df[df["PateID"].isin(random_patient)]

df_sample = df_sample.sort_values(by=["PateID"])
df_sample.to_excel("./test.xlsx")


### Vytvorenie noveho datasetu
- Podla clankov spravim priemer hodnot, upravim vek na int, onehot encoding ak je potrebne a zaroven necham 0/1 data
- `EXAM`: mozem z toho podom delit lebo to je pocet vysetreni, nemusim robit count ci podobne si uchovatat pocet stretnuti

In [58]:
binary_cols: list[str] = ["Gender", "Atopy", "Eczema", "Hayfever",
                             "Asthma", "Allergies", "Rubbing", "EXAM"
                             ]

ignore_cols: list[str] = ["Exam_date", "DOB", "referred_CXL", "PateID"]

all_cols: list[str] = df.columns.tolist()

measurement_cols = [
    c for c in all_cols
    if c not in binary_cols + ignore_cols
]



cleaned_df = (
    df
    .sort_values("Exam_date")
    .groupby("PateID")
    .agg(
        {**{c: "max" for c in binary_cols},
         **{c: "mean" for c in measurement_cols},
         "Exam_date": "mean",
         "DOB": "first",
         "referred_CXL": "last"
         }
    )
    .reset_index()
)

cleaned_df["Age"] = (
    (cleaned_df["Exam_date"] - cleaned_df["DOB"]).dt.days / 365.25      # type: ignore
).round(1)

cleaned_df = cleaned_df.drop(columns=["Exam_date", "DOB"])

#### NaN pre referred_CXL

In [59]:
# 1)
df["referred_CXL"] = df["referred_CXL"].fillna(0)

# 2)
# ondstranit riadky (pacientov) pre ktorych nebolo rozhodnute
#! [PRIDAT KOLACOVI GRAF NA POROVNANIE]

In [60]:
cleaned_df.to_excel("./clean_ds.xlsx")